# CVEFixes by language

## Retrieve dataset from HF

In [2]:
import pandas as pd
from datasets import load_dataset
import os

os.environ["HF_HOME"] = "/tmp/huggingface"
# dataset = load_dataset("euisuh15/cveFixes1")
dataset = load_dataset("euisuh15/cveFixes1", split='c')

# lang option:  c | go | java | python | ruby
# lang = "c"
# df = pd.DataFrame(dataset[lang])
df = pd.DataFrame(dataset)

df.shape

/home/local/QCRI/ejeong/anaconda2/envs/vllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(2621, 14)

In [3]:
df.head()

,file_name,method_name,code_before,code_after,func_before,func_after,diff,num_lines_added,num_lines_deleted,num_lines_in_file,num_tokens_in_file,repo,cve_id,cwe_id
0,open_utils.c,read_new_config_info,//////////////////////////////////////////////...,//////////////////////////////////////////////...,static int read_new_config_info (WavpackContex...,static int read_new_config_info (WavpackContex...,"{'added': [(563, ' if (bytecnt >= 2) {'), (...",5.0,1.0,825.0,6305.0,https://github.com/dbry/WavPack,CVE-2016-10169,['CWE-125']
1,route.c,inet_rtm_getroute,/*\n * INET\t\tAn implementation of the TCP/IP...,/*\n * INET\t\tAn implementation of the TCP/IP...,static int inet_rtm_getroute(struct sk_buff *i...,static int inet_rtm_getroute(struct sk_buff *i...,"{'added': [(2766, '\tif (rtm->rtm_flags & RTM_...",9.0,2.0,2327.0,14667.0,https://github.com/torvalds/linux,CVE-2017-13686,['CWE-476']
2,main.c,mp4boxMain,/*\n *\t\t\tGPAC - Multimedia Framework C SDK\...,/*\n *\t\t\tGPAC - Multimedia Framework C SDK\...,"int mp4boxMain(int argc, char **argv)\n{\n\tu3...","int mp4boxMain(int argc, char **argv)\n{\n\tu3...","{'added': [(5598, '\t\t\tfprintf(stderr, ""File...",1.0,1.0,5787.0,43960.0,https://github.com/gpac/gpac,CVE-2020-23930,['CWE-476']
3,udp.c,udpv6_sendmsg,/*\n *\tUDP over IPv6\n *\tLinux INET6 impleme...,/*\n *\tUDP over IPv6\n *\tLinux INET6 impleme...,"int udpv6_sendmsg(struct sock *sk, struct msgh...","int udpv6_sendmsg(struct sock *sk, struct msgh...","{'added': [(1113, '\tstruct ipv6_txoptions *op...",6.0,2.0,1221.0,7876.0,https://github.com/torvalds/linux,CVE-2016-3841,"['CWE-264', 'CWE-416']"
4,api.c,crypto_larval_lookup,/*\n * Scatterlist Cryptographic API.\n *\n * ...,/*\n * Scatterlist Cryptographic API.\n *\n * ...,struct crypto_alg *crypto_larval_lookup(const ...,struct crypto_alg *crypto_larval_lookup(const ...,"{'added': [(219, '\t\trequest_module(""crypto-%...",2.0,2.0,424.0,2423.0,https://github.com/torvalds/linux,CVE-2013-7421,['CWE-269']


In [4]:
df.to_csv('vul7.csv')